# EM-DAT

In [1]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

In [4]:
import ocha_stratus as stratus
import pandas as pd
import geopandas as gpd
import xarray as xr
from tqdm.auto import tqdm
from dask.diagnostics import ProgressBar
from rasterio.errors import RasterioIOError

from src.datasources import codab, imerg, emdat
from src.constants import *

In [5]:
adm0 = codab.load_codab_from_blob()

In [6]:
df_emdat_new = emdat.load_emat()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [7]:
query = """
SELECT *
FROM storms.ibtracs_storms
"""
with stratus.get_engine(stage="prod").connect() as con:
    df_storms = pd.read_sql(query, con)

In [8]:
blob_name = f"{PROJECT_PREFIX}/processed/emdat-tropicalcyclone-2000-2022-processed-sids.csv"
df_emdat_old_raw = stratus.load_csv_from_blob(blob_name)

In [9]:
df_emdat_old = df_emdat_old_raw[
    df_emdat_old_raw["iso3"] == ISO3.upper()
].copy()

In [10]:
df_emdat_old.columns

Index(['DisNo.', 'Historic', 'Classification Key', 'Disaster Group',
       'Disaster Subgroup', 'Disaster Type', 'Disaster Subtype',
       'External IDs', 'Event Name', 'iso3', 'Country', 'Subregion', 'Region',
       'Location', 'Origin', 'Associated Types', 'OFDA Response', 'Appeal',
       'Declaration', 'AID Contribution ('000 US$)', 'Magnitude',
       'Magnitude Scale', 'Latitude', 'Longitude', 'River Basin', 'Start Year',
       'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day',
       'Total Deaths', 'No. Injured', 'No. Affected', 'No. Homeless',
       'Total Affected', 'Reconstruction Costs ('000 US$)',
       'Reconstruction Costs, Adjusted ('000 US$)',
       'Insured Damage ('000 US$)', 'Insured Damage, Adjusted ('000 US$)',
       'Total Damage ('000 US$)', 'Total Damage, Adjusted ('000 US$)', 'CPI',
       'Admin Units', 'Entry Date', 'Last Update', 'iso2', 'asap0_id', 'sid'],
      dtype='object')

In [11]:
# see which ones never had SIDs filled in
df_emdat_old[df_emdat_old["sid"].isnull()][
    [
        "DisNo.",
        "Event Name",
        "Start Year",
        "Start Month",
        "Start Day",
        "Total Affected",
    ]
]

,DisNo.,Event Name,Start Year,Start Month,Start Day,Total Affected
125,2003-0823-PHL,Ineng,2003,7,30.0,3748.0
132,2004-0609-PHL,Winnie,2004,11,29.0,881023.0
269,2022-0204-PHL,Tropical storm 'Megi' (Agaton),2022,4,10.0,2298788.0
270,2022-0547-PHL,Tropical Cyclone 'Ma-On' (Florita),2022,8,22.0,7616.0
271,2022-0665-PHL,Tropical depression 'Maymay',2022,10,13.0,NaN
272,2022-0680-PHL,Tropical cyclone 'Nesat',2022,10,15.0,103602.0
273,2022-0707-PHL,Storm 'Nalgae' (Paeng),2022,10,27.0,3323291.0


In [12]:
# get values for storms not in IBTrACS (quick Google)
df_missing_ibtracs = pd.DataFrame(
    columns=[
        "name",
        "season",
        "DisNo.",
        "start_date",
        "end_date",
        "wind_speed_max_kmh",
    ],
    data=[
        ("INENG", 2003, "2003-0823-PHL", "2003-07-30", "2003-07-31", 45),
        ("WINNIE", 2004, "2004-0609-PHL", "2004-11-29", "2004-11-30", 55),
        ("MAYMAY", 2022, "2022-0665-PHL", "2022-10-11", "2022-10-12", 55),
    ],
)

In [13]:
# get matches from IBTrACS for missing rows
disno2sid_old = {
    "2022-0204-PHL": "2022099N11128",
    "2022-0547-PHL": "2022232N18131",
    "2022-0680-PHL": "2022285N17140",
    "2022-0707-PHL": "2022299N11134",
}

In [14]:
df_emdat_old["sid"] = df_emdat_old["sid"].fillna(
    df_emdat_old["DisNo."].apply(lambda x: disno2sid_old.get(x))
)

In [15]:
df_emdat_old[df_emdat_old["sid"].isnull()][
    ["DisNo.", "Event Name", "Start Year", "Start Month", "Start Day"]
]

,DisNo.,Event Name,Start Year,Start Month,Start Day
125,2003-0823-PHL,Ineng,2003,7,30.0
132,2004-0609-PHL,Winnie,2004,11,29.0
271,2022-0665-PHL,Tropical depression 'Maymay',2022,10,13.0


In [16]:
# see which new EM-DAT events aren't in the old matched up CSV file
df_emdat_new[~df_emdat_new["DisNo."].isin(df_emdat_old["DisNo."].values)][
    ["DisNo.", "Event Name", "Start Year", "Start Month", "Start Day"]
]

,DisNo.,Event Name,Start Year,Start Month,Start Day
166,2023-0246-PHL,Tropical Depression 'Amang',2023,4.0,17.0
167,2023-0300-PHL,Typhoon 'Mawar',2023,5.0,24.0
168,2023-0450-PHL,Tropical cyclone 'Talim',2023,7.0,12.0
169,2023-0464-PHL,Tropical cyclone 'Doksuri' (Egay),2023,7.0,21.0
170,2023-0568-PHL,Typhoon 'Saola',2023,8.0,22.0
171,2023-0623-PHL,Typhoon 'Koinu',2023,10.0,4.0
172,2023-0842-PHL,Tropical storm 'Jelawat' (Kabayan),2023,12.0,18.0
173,2024-0337-PHL,Tropical cyclone 'Ewiniar' (Aghon),2024,5.0,24.0
174,2024-0522-PHL,Typhoon 'Gaemi' (Carina) and 'Butchoy' (Prapir...,2024,7.0,22.0
175,2024-0654-PHL,Typhoon 'Yagi' (Enteng),2024,9.0,1.0


In [17]:
# limit to only before this year
df_emdat_new_recent = df_emdat_new[
    (~df_emdat_new["DisNo."].isin(df_emdat_old["DisNo."].values))
    & (df_emdat_new["Start Year"] < 2025)
].copy()
df_emdat_new_recent[
    ["DisNo.", "Event Name", "Start Year", "Start Month", "Start Day"]
]

,DisNo.,Event Name,Start Year,Start Month,Start Day
166,2023-0246-PHL,Tropical Depression 'Amang',2023,4.0,17.0
167,2023-0300-PHL,Typhoon 'Mawar',2023,5.0,24.0
168,2023-0450-PHL,Tropical cyclone 'Talim',2023,7.0,12.0
169,2023-0464-PHL,Tropical cyclone 'Doksuri' (Egay),2023,7.0,21.0
170,2023-0568-PHL,Typhoon 'Saola',2023,8.0,22.0
171,2023-0623-PHL,Typhoon 'Koinu',2023,10.0,4.0
172,2023-0842-PHL,Tropical storm 'Jelawat' (Kabayan),2023,12.0,18.0
173,2024-0337-PHL,Tropical cyclone 'Ewiniar' (Aghon),2024,5.0,24.0
174,2024-0522-PHL,Typhoon 'Gaemi' (Carina) and 'Butchoy' (Prapir...,2024,7.0,22.0
175,2024-0654-PHL,Typhoon 'Yagi' (Enteng),2024,9.0,1.0


In [18]:
# match to IBTrACS
disno2sid_new = {
    "2023-0246-PHL": "2023101N14127",
    "2023-0300-PHL": "2023138N05151",
    "2023-0450-PHL": "2023194N16123",
    "2023-0464-PHL": "2023201N13134",
    "2023-0568-PHL": "2023234N18128",
    "2023-0623-PHL": "2023271N14144",
    "2023-0842-PHL": "2023349N04141",
    "2024-0337-PHL": "2024141N03142",
    "2024-0522-PHL": "2024201N12133",
    "2024-0654-PHL": "2024244N09137",
    "2024-0666-PHL": "2024253N11148",
    "2024-0695-PHL": "2024259N17126",
    "2024-0711-PHL": "2024270N24128",
    "2024-0777-PHL": "2024293N13141",
    "2024-0801-PHL": "2024298N13150",
    "2024-0825-PHL": "2024307N06143",
    "2024-0829-PHL": "2024312N14145",
    "2024-0855-PHL": "2024313N10169",
}

In [19]:
df_emdat_new_recent["sid"] = df_emdat_new_recent["DisNo."].replace(
    disno2sid_new
)

In [20]:
df_emdat_new_recent[df_emdat_new_recent["sid"].isnull()]

,DisNo.,Historic,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,External IDs,Event Name,ISO,...,"Reconstruction Costs, Adjusted ('000 US$)",Insured Damage ('000 US$),"Insured Damage, Adjusted ('000 US$)",Total Damage ('000 US$),"Total Damage, Adjusted ('000 US$)",CPI,Admin Units,Entry Date,Last Update,sid


In [21]:
# aggregate rainfall for storms missing from IBTrACS
quantiles = [0.8, 0.9, 0.95]


def get_storm_rainfall_aggregations(row):
    row = row.copy()
    min_date = row["valid_time_min"].date() - pd.DateOffset(days=1)
    max_date = row["valid_time_max"].date() + pd.DateOffset(days=1)
    dates = pd.date_range(min_date, max_date)
    da = imerg.open_imerg_raster_dates(dates)
    da_clip = da.rio.clip(adm0.geometry)

    # 2-day rolling sum
    da_rolling2 = da_clip.rolling(date=2).sum()
    # 3-day rolling sum
    da_rolling3 = da_clip.rolling(date=3).sum()

    # take quantiles
    for quantile in quantiles:
        for da_agg, agg_str in [
            (da_rolling2, "roll2"),
            (da_rolling3, "roll3"),
        ]:
            # get quantile threshs
            quantile_threshs = da_agg.quantile(quantile, dim=["x", "y"])
            # get max value
            row[f"q{quantile*100:.0f}_{agg_str}"] = float(
                quantile_threshs.max()
            )

    return row


import warnings

warnings.filterwarnings("ignore", message="All-NaN slice encountered")

In [22]:
df_missing_ibtracs["valid_time_min"] = pd.to_datetime(
    df_missing_ibtracs["start_date"]
)
df_missing_ibtracs["valid_time_max"] = pd.to_datetime(
    df_missing_ibtracs["end_date"]
)
df_missing_ibtracs["wind_speed_max"] = (
    df_missing_ibtracs["wind_speed_max_kmh"] / KNOTS_TO_KMH
).astype(int)

In [23]:
df_missing_ibtracs

,name,season,DisNo.,start_date,end_date,wind_speed_max_kmh,valid_time_min,valid_time_max,wind_speed_max
0,INENG,2003,2003-0823-PHL,2003-07-30,2003-07-31,45,2003-07-30,2003-07-31,24
1,WINNIE,2004,2004-0609-PHL,2004-11-29,2004-11-30,55,2004-11-29,2004-11-30,29
2,MAYMAY,2022,2022-0665-PHL,2022-10-11,2022-10-12,55,2022-10-11,2022-10-12,29


In [24]:
tqdm.pandas()

In [ ]:
df_missing_ibtracs_with_stats = df_missing_ibtracs.progress_apply(
    get_storm_rainfall_aggregations, axis=1
)

In [26]:
# create madeup SID to make merging easier
df_missing_ibtracs_with_stats["sid"] = (
    df_missing_ibtracs_with_stats["name"].str.lower()
    + "_"
    + df_missing_ibtracs_with_stats["season"].astype(str)
)

In [27]:
df_missing_ibtracs_with_stats

,name,season,DisNo.,start_date,end_date,wind_speed_max_kmh,valid_time_min,valid_time_max,wind_speed_max,q80_roll2,q80_roll3,q90_roll2,q90_roll3,q95_roll2,q95_roll3,sid
0,INENG,2003,2003-0823-PHL,2003-07-30,2003-07-31,45,2003-07-30,2003-07-31,24,43.145000,45.988998,56.284000,60.473499,70.509247,73.251755,ineng_2003
1,WINNIE,2004,2004-0609-PHL,2004-11-29,2004-11-30,55,2004-11-29,2004-11-30,29,65.145996,105.078995,135.749496,150.975006,202.641235,227.532745,winnie_2004
2,MAYMAY,2022,2022-0665-PHL,2022-10-11,2022-10-12,55,2022-10-11,2022-10-12,29,36.324997,43.250996,79.607002,94.037491,168.749512,189.808243,maymay_2022


In [28]:
df_missing_ibtracs_with_stats_with_emdat = df_missing_ibtracs_with_stats.merge(
    df_emdat_old.drop(columns="sid")
)

In [29]:
df_missing_ibtracs_with_stats_with_emdat

,name,season,DisNo.,start_date,end_date,wind_speed_max_kmh,valid_time_min,valid_time_max,wind_speed_max,q80_roll2,...,Insured Damage ('000 US$),"Insured Damage, Adjusted ('000 US$)",Total Damage ('000 US$),"Total Damage, Adjusted ('000 US$)",CPI,Admin Units,Entry Date,Last Update,iso2,asap0_id
0,INENG,2003,2003-0823-PHL,2003-07-30,2003-07-31,45,2003-07-30,2003-07-31,24,43.145000,...,NaN,NaN,146.0,232.0,62.858460,"[{""adm1_code"":2361,""adm1_name"":""Region V (Bico...",2006-04-28,2023-09-25,PH,126
1,WINNIE,2004,2004-0609-PHL,2004-11-29,2004-11-30,55,2004-11-29,2004-11-30,29,65.145996,...,NaN,NaN,78200.0,121163.0,64.541329,"[{""adm2_code"":24217,""adm2_name"":""Isabela""},{""a...",2005-07-28,2023-09-25,PH,126
2,MAYMAY,2022,2022-0665-PHL,2022-10-11,2022-10-12,55,2022-10-11,2022-10-12,29,36.324997,...,NaN,NaN,NaN,NaN,100.000000,NaN,2022-10-13,2023-09-26,PH,126


In [30]:
df_emdat_combined = pd.concat(
    [df_emdat_old, df_emdat_new_recent], ignore_index=True
)

In [269]:
# df_emdat_combined["sid"] = df_emdat_combined.apply(
#     lambda row: (
#         row["sid"]
#         if row["sid"]
#         else df_missing_ibtracs_with_stats.set_index("DisNo.").loc[
#             row["DisNo."], "sid"
#         ]
#     ),
#     axis=1,
# )

In [31]:
df_emdat_combined[df_emdat_combined["sid"].isnull()]

,DisNo.,Historic,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,External IDs,Event Name,iso3,...,"Total Damage, Adjusted ('000 US$)",CPI,Admin Units,Entry Date,Last Update,iso2,asap0_id,sid,ISO,OFDA/BHA Response
17,2003-0823-PHL,No,nat-met-sto-tro,Natural,Meteorological,Storm,Tropical cyclone,NaN,Ineng,PHL,...,232.0,62.858460,"[{""adm1_code"":2361,""adm1_name"":""Region V (Bico...",2006-04-28,2023-09-25,PH,126.0,None,NaN,NaN
24,2004-0609-PHL,No,nat-met-sto-tro,Natural,Meteorological,Storm,Tropical cyclone,NaN,Winnie,PHL,...,121163.0,64.541329,"[{""adm2_code"":24217,""adm2_name"":""Isabela""},{""a...",2005-07-28,2023-09-25,PH,126.0,None,NaN,NaN
163,2022-0665-PHL,No,nat-met-sto-tro,Natural,Meteorological,Storm,Tropical cyclone,NaN,Tropical depression 'Maymay',PHL,...,NaN,100.000000,NaN,2022-10-13,2023-09-26,PH,126.0,None,NaN,NaN


In [32]:
blob_name = f"{PROJECT_PREFIX}/processed/ibtracs_imerg_stats_50km.parquet"
df_stats_raw = stratus.load_parquet_from_blob(blob_name)

In [33]:
df_stats_raw

,sid,valid_time_min,valid_time_max,wind_speed_max,q80_roll2,q80_roll3,q90_roll2,q90_roll3,q95_roll2,q95_roll3
0,2000248N18126,2000-09-10 06:00:00,2000-09-11 12:00:00,45.0,45.054001,58.357998,88.719994,98.764496,122.616997,136.360245
1,2000299N08139,2000-10-27 18:00:00,2000-10-28 18:00:00,60.0,165.409988,209.722000,224.842987,290.563507,279.513733,349.203461
2,2000305N06136,2000-11-02 06:00:00,2000-11-03 00:00:00,55.0,104.563995,131.760010,203.577957,229.390518,294.647766,338.187714
3,2001125N05129,2001-05-09 12:00:00,2001-05-13 00:00:00,50.0,75.133995,97.445000,119.209991,137.260498,160.666977,176.315735
4,2001170N11138,2001-06-22 06:00:00,2001-06-22 06:00:00,60.0,66.177994,81.712006,96.375496,123.316994,136.060989,157.362991
...,...,...,...,...,...,...,...,...,...,...
130,2025267N10134,2025-09-25 18:00:00,2025-09-26 06:00:00,60.0,97.699997,105.072998,152.252502,161.379501,178.669998,186.904739
131,2025274N15131,2025-10-03 00:00:00,2025-10-03 06:00:00,64.0,88.656998,91.140991,106.903000,109.906982,120.015488,123.223999
132,2025291N13126,2025-10-18 06:00:00,2025-10-19 06:00:00,35.0,84.789986,93.028000,113.498497,123.448509,159.110992,167.795258
133,2025305N10138,2025-11-03 12:00:00,2025-11-05 00:00:00,89.0,118.521011,130.082993,149.364990,159.100006,180.468491,185.976715


In [34]:
df_stats_with_ibtracs = df_stats_raw.merge(
    df_emdat_combined, how="left"
).merge(df_storms)

In [35]:
df_stats = pd.concat(
    [df_stats_with_ibtracs, df_missing_ibtracs_with_stats_with_emdat],
    ignore_index=True,
)

In [ ]:
blob_name = (
    f"{PROJECT_PREFIX}/processed/ibtracs_imerg_emdat_stats_50km.parquet"
)
stratus.upload_parquet_to_blob(df_stats, blob_name)